In [1]:
from ahocorapy.keywordtree import KeywordTree

In [2]:
kwtree1 = KeywordTree(case_insensitive=True)
kwtree1.add('smoke')
kwtree1.add('smoking')
kwtree1.add('tobacco')
kwtree1.add('cigarette')
kwtree1.add('inhale')
kwtree1.finalize()

kwtree2 = KeywordTree(case_insensitive=True)
kwtree2.add('fast food')
kwtree2.add('burgers')
kwtree2.add('mcdonalds')
kwtree2.add('sandwich')
kwtree2.add('fries')
kwtree2.finalize()

In [7]:
result1 = kwtree1.search('my tummy hurts i wonder if the hypnosis has anything to do with it if so its working i get it stop smoking')
print(result1)

result2 = kwtree2.search('my tummy hurts i wonder if the hypnosis has anything to do with it if so its working i get it stop smoking')
print(result2)

('smoking', 99)
None


In [8]:
results1 = kwtree1.search_all('my tummy hurts i wonder if the hypnosis has anything to do with it if so its working i get it stop smoking')
for result in results1:
    print(result)
    
results2 = kwtree2.search_all('my tummy hurts i wonder if the hypnosis has anything to do with it if so its working i get it stop smoking')
for result in results2:
    print(result)

('smoking', 99)


### Topic analysis

In [2]:
def KeywordWeight(searchResult, words, weights):
    keyword = []
    keyword_weight = []
    for result in searchResult:
        keyword.append(result[0])
    for k in keyword:
        for i in range(len(words)):
            if words[i] == k:
                keyword_weight.append(weights[i])
                break
    return keyword, keyword_weight
        
def scoring(keyword, weight, real_sentiment, intended_sentiment, sentilevel):
    score = 0
    for i in range(len(weight)):
        score += float(weight[i])
    if real_sentiment == intended_sentiment:
        discount = sentilevel
    else:
        discount = -sentilevel
    score = score * discount
    return score

def searcher(filepath):
    kwtree_word = []
    kwtree_weight = []
    f = open(filepath)
    for line in f:
        word, weight = line.split(' ')
        word = word.replace('_', ' ')
        kwtree_word.append(word)
        weight = weight.split('\n')[0]
        kwtree_weight.append(weight)
    f.close()
    kwtree = KeywordTree(case_insensitive=True)
    for word in kwtree_word:
        kwtree.add(word)
    kwtree.finalize()
    return kwtree, kwtree_word, kwtree_weight

In [3]:
import sys
sys.path.append('..\\sentiment analysis\\release')
import sentiment_prediction as senti

tweet1 = "i love junk food beacuse they are cheap and delicious."

modelpath = 'D:\\COMP90024_Assignment2\\sentiment analysis\\sentiment_lstm.h5'
pklpath = 'D:\\COMP90024_Assignment2\\sentiment analysis\\training_data\\large\\sentiment140-freqdist.pkl'
analysor = senti.sentianalysor(modelpath, pklpath, stemmer = False)

dictionary = '.\\fastfood.txt'
fastfood_searcher, words, weight = searcher(dictionary)

search_result = fastfood_searcher.search_all(tweet1)
keyword, keyword_weight = KeywordWeight(search_result, words, weight)
print('Keyword:', keyword)
print('Weight:', keyword_weight)
prediction = analysor.prediction(tweet1)
real_senti = prediction[0]
sentilevel = prediction[1]
intended_sentiment = 'POSITIVE'

score =  score = scoring(keyword, keyword_weight, intended_sentiment, real_senti, sentilevel)
print('score:', score)

Using TensorFlow backend.


Keyword: ['junk food']
Weight: ['1']
score: 0.9711006283760071


In [59]:
tweet = [
    "I am the fan of burger king cos its really delicious!",
    "I don't like burgers since i think they are unhealthy.",
    "Well i think i'll go to have a cup of coffee."
]

for i in range(len(tweet)):
    t = tweet[i]
    search_result = fastfood_searcher.search_all(t)
    keyword, keyword_weight = KeywordWeight(search_result, words, weight)
    prediction = analysor.prediction(t)
    real_senti = prediction[0]
    sentilevel = prediction[1]
    intended_sentiment = 'POSITIVE'
    score =  score = scoring(keyword, keyword_weight, intended_sentiment, real_senti, sentilevel)
    print('score:', score)

score: 2.6899623579633576
score: -1.811632689465635
score: 0.21618476496974232


### Let's get multiple searchers!

In [4]:
fastfood_dictionary = '.\\fastfood.txt'
fastfood_searcher, fastfood_words, fastfood_weight = searcher(fastfood_dictionary)

smoking_dictionary = '.\\smoking.txt'
smoking_searcher, smoking_words, smoking_weight = searcher(smoking_dictionary)

alcohols_dictionary = '.\\alcohols.txt'
alcohols_searcher, alcohols_words, alcohols_weight = searcher(alcohols_dictionary)

modelpath = 'D:\\COMP90024_Assignment2\\sentiment analysis\\sentiment_lstm.h5'
pklpath = 'D:\\COMP90024_Assignment2\\sentiment analysis\\training_data\\large\\sentiment140-freqdist.pkl'
analysor = senti.sentianalysor(modelpath, pklpath, stemmer = False)

In [5]:
tweet1 = 'I am the fan of burger king cos its really delicious!'

intended_sentiment = 'POSITIVE'

prediction = analysor.prediction(tweet1)
real_senti = prediction[0]
sentilevel = prediction[1]

fd_result = fastfood_searcher.search_all(tweet1)
fd_keyword, fd_keyword_weight = KeywordWeight(fd_result, fastfood_words, fastfood_weight)
fd_score = scoring(fd_keyword, fd_keyword_weight, intended_sentiment, real_senti, sentilevel)
print('fastfood score:', fd_score)

smoke_result = smoking_searcher.search_all(tweet1)
smoke_keyword, smoke_keyword_weight = KeywordWeight(smoke_result, smoking_words, smoking_weight)
smoke_score = scoring(smoke_keyword, smoke_keyword_weight, intended_sentiment, real_senti, sentilevel)
print('smoking score:', smoke_score)

alcohols_result = alcohols_searcher.search_all(tweet1)
alcohols_keyword, alcohols_keyword_weight = KeywordWeight(alcohols_result, alcohols_words, alcohols_weight)
alcohols_score = scoring(alcohols_keyword, alcohols_keyword_weight, intended_sentiment, real_senti, sentilevel)
print('alcohols score:', alcohols_score)

fastfood score: 1.8664179067015647
smoking score: 0.0
alcohols score: 0.0


In [6]:
tweet2 = 'I wanna smoke which makes me feeling sooo good!'

intended_sentiment = 'POSITIVE'

prediction = analysor.prediction(tweet2)
real_senti = prediction[0]
sentilevel = prediction[1]

fd_result = fastfood_searcher.search_all(tweet2)
fd_keyword, fd_keyword_weight = KeywordWeight(fd_result, fastfood_words, fastfood_weight)
fd_score = scoring(fd_keyword, fd_keyword_weight, intended_sentiment, real_senti, sentilevel)
print('fastfood score:', fd_score)

smoke_result = smoking_searcher.search_all(tweet2)
smoke_keyword, smoke_keyword_weight = KeywordWeight(smoke_result, smoking_words, smoking_weight)
smoke_score = scoring(smoke_keyword, smoke_keyword_weight, intended_sentiment, real_senti, sentilevel)
print('smoking score:', smoke_score)

alcohols_result = alcohols_searcher.search_all(tweet2)
alcohols_keyword, alcohols_keyword_weight = KeywordWeight(alcohols_result, alcohols_words, alcohols_weight)
alcohols_score = scoring(alcohols_keyword, alcohols_keyword_weight, intended_sentiment, real_senti, sentilevel)
print('alcohols score:', alcohols_score)

fastfood score: 0.0
smoking score: 0.7204580812454223
alcohols score: 0.0


In [7]:
tweet3 = 'HAPPY HOURS AGAIN! definately i will go to pub and have a cup of whiskey tonight!'

intended_sentiment = 'POSITIVE'

prediction = analysor.prediction(tweet3)
real_senti = prediction[0]
sentilevel = prediction[1]

fd_result = fastfood_searcher.search_all(tweet3)
fd_keyword, fd_keyword_weight = KeywordWeight(fd_result, fastfood_words, fastfood_weight)
fd_score = scoring(fd_keyword, fd_keyword_weight, intended_sentiment, real_senti, sentilevel)
print('fastfood score:', fd_score)

smoke_result = smoking_searcher.search_all(tweet3)
smoke_keyword, smoke_keyword_weight = KeywordWeight(smoke_result, smoking_words, smoking_weight)
smoke_score = scoring(smoke_keyword, smoke_keyword_weight, intended_sentiment, real_senti, sentilevel)
print('smoking score:', smoke_score)

alcohols_result = alcohols_searcher.search_all(tweet3)
alcohols_keyword, alcohols_keyword_weight = KeywordWeight(alcohols_result, alcohols_words, alcohols_weight)
alcohols_score = scoring(alcohols_keyword, alcohols_keyword_weight, intended_sentiment, real_senti, sentilevel)
print('alcohols score:', alcohols_score)

fastfood score: 0.0
smoking score: 0.0
alcohols score: 0.9682081018304826


In [24]:
def topicker(tweet, topics, topic_dict_path, intended_sentiments, analysor):
    
    def scores_checker(scores):
        Flag = True
        if max(scores) != 0.0:            
            for i in range(len(scores)):
                if scores[i] != 0.0:
                    Flag = False
                    break
        return Flag
    
    scores = []
    prediction = analysor.prediction(tweet)
    real_senti = prediction[0]
    sentilevel = prediction[1]
    for i in range(len(topic_dict_path)):
        intended_sentiment = intended_sentiments[i]
        dictionary = topic_dict_path[i]
        this_searcher, dict_words, dict_weights = searcher(dictionary)
        search_result = this_searcher.search_all(tweet)
        keywords, keywords_weights = KeywordWeight(search_result, dict_words, dict_weights)
        scores.append(scoring(keywords, keywords_weights, intended_sentiment, real_senti, sentilevel))
    if scores_checker(scores):
        matched_topic = 'NONE OF THESE'
        return matched_topic
    else:
        max_index = scores.index(max(scores))
        matched_topic = topics[max_index]
        return matched_topic

In [35]:
tweet = "smoking makes me feeling better."

topics = ['fast food', 'smoking', 'alcohols']
topics_dictionary_path = ['fastfood.txt', 'smoking.txt', 'alcohols.txt']
intended_sentiments = ['POSITIVE', 'POSITIVE', 'POSITIVE']
modelpath = 'D:\\COMP90024_Assignment2\\sentiment analysis\\sentiment_lstm.h5'
pklpath = 'D:\\COMP90024_Assignment2\\sentiment analysis\\training_data\\large\\sentiment140-freqdist.pkl'
analysor = senti.sentianalysor(modelpath, pklpath, stemmer = False)

matched = topicker(tweet, topics, topics_dictionary_path, intended_sentiments, analysor)
print('Topic matched:', matched)

Topic matched: smoking
